In [4]:
import pandas as pd
import numpy as np
from acquire import get_zillow

In [2]:
# getting df
df = get_zillow()

In [6]:
# getting shape
df.shape

(77579, 77)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77579 entries, 0 to 77578
Data columns (total 77 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77579 non-null  int64  
 1   parcelid                      77579 non-null  int64  
 2   airconditioningtypeid         25007 non-null  float64
 3   architecturalstyletypeid      207 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   77579 non-null  float64
 6   bedroomcnt                    77579 non-null  float64
 7   buildingclasstypeid           15 non-null     float64
 8   buildingqualitytypeid         49809 non-null  float64
 9   calculatedbathnbr             76963 non-null  float64
 10  decktypeid                    614 non-null    float64
 11  finishedfloor1squarefeet      6037 non-null   float64
 12  calculatedfinishedsquarefeet  77378 non-null  float64
 13  f

In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,77579.0,1.495392e+06,8.609686e+05,349.0,752142.0,1498227.0,2240879.0,2982274.0
parcelid,77579.0,1.300828e+07,3.519399e+06,10711855.0,11538198.0,12530543.0,14211357.5,167689317.0
airconditioningtypeid,25007.0,1.812013e+00,2.965768e+00,1.0,1.0,1.0,1.0,13.0
architecturalstyletypeid,207.0,7.386473e+00,2.728030e+00,2.0,7.0,7.0,7.0,21.0
basementsqft,50.0,6.797200e+02,6.897035e+02,38.0,273.0,515.0,796.5,3560.0
...,...,...,...,...,...,...,...,...
buildingclasstypeid,15.0,3.933333e+00,2.581989e-01,3.0,4.0,4.0,4.0,4.0
heatingorsystemtypeid,49571.0,3.921749e+00,3.594780e+00,1.0,2.0,2.0,7.0,24.0
propertylandusetypeid,77579.0,2.618245e+02,5.141596e+00,31.0,261.0,261.0,266.0,275.0
storytypeid,50.0,7.000000e+00,0.000000e+00,7.0,7.0,7.0,7.0,7.0


In [74]:
# looking at duplicated parcel id's
df.duplicated(subset=['parcelid']).value_counts()

False    77226
True       197
dtype: int64

In [82]:
# replacing duplicates and keeping the first
df = df.sort_values('transactiondate', ascending=False).drop_duplicates(subset=['parcelid'], keep='first')

In [85]:
df.duplicated()

77578    False
77577    False
77576    False
77575    False
77574    False
         ...  
5        False
4        False
3        False
2        False
0        False
Length: 77226, dtype: bool

In [87]:
def getDuplicateColumns(df):

	# Create an empty set
	duplicateColumnNames = set()

	# Iterate through all the columns
	# of dataframe
	for x in range(df.shape[1]):

		# Take column at xth index.
		col = df.iloc[:, x]

		# Iterate through all the columns in
		# DataFrame from (x + 1)th index to
		# last index
		for y in range(x + 1, df.shape[1]):

			# Take column at yth index.
			otherCol = df.iloc[:, y]

			# Check if two columns at x & y
			# index are equal or not,
			# if equal then adding
			# to the set
			if col.equals(otherCol):
				duplicateColumnNames.add(df.columns.values[y])

	# Return list of unique column names
	# whose contents are duplicates.
	return list(duplicateColumnNames)


In [88]:
getDuplicateColumns(df)

['parcelid', 'heatingorsystemtypeid']

In [11]:
def remove_columns(df, cols_to_remove):  
    df = df.drop(columns=cols_to_remove)
    return df

In [25]:
def handle_missing_values(df, prop_required_col = .5, prop_required_row = .75):
    '''
    Actions: this function removes columns and rows based on the proportion of data missing
    
    '''
    # COL
    # setting a threshold for columns based on kwarg
    threshold = int(round(prop_required_col*len(df.index), 0))
    
    # dropping the null values that meet the threshold established
    df.dropna(axis=1, thresh=threshold, inplace=True)
    
    # ROW
    # setting a threshold for the rows based on the kwarg
    threshold = int(round(prop_required_row*len(df.columns), 0))
    
    # dropping the null values that meet the threshold established
    df.dropna(axis=0, thresh=threshold, inplace=True)
    
    return df

In [20]:
# setting proportion
prop_required_col = .5

# creating a threshold amount 
threshold = int(round(prop_required_col*len(df.index), 0))
print(threshold)

38790


In [23]:
len((df.dropna(axis=1, thresh=threshold)).columns)

38

In [26]:
# removing missing values now
df_prep = handle_missing_values(df)

In [28]:
df_prep.shape

(77423, 38)